In [2]:
! pip install shioaji

  Using cached shioaji-0.1.2.dev9-cp37-cp37m-manylinux1_x86_64.whl (4.4 MB)
  Using cached requests-2.22.0-py2.py3-none-any.whl (57 kB)
  Using cached xxhash-1.4.3-cp37-cp37m-manylinux2010_x86_64.whl (234 kB)
  Using cached pydantic-1.0-cp37-cp37m-manylinux1_x86_64.whl (5.7 MB)
  Using cached sentry_sdk-0.14.1-py2.py3-none-any.whl (93 kB)
  Using cached pysolace-0.2.5-cp37-cp37m-manylinux2010_x86_64.whl (5.9 MB)
  Using cached loguru-0.4.1-py3-none-any.whl (54 kB)
  Using cached idna-2.8-py2.py3-none-any.whl (58 kB)
     |████████████████████████████████| 275 kB 1.1 MB/s eta 0:00:01
  Attempting uninstall: idna
    Found existing installation: idna 2.9
    Uninstalling idna-2.9:
      Successfully uninstalled idna-2.9
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [1]:
import json
import datetime
import shioaji as sj
import dolphindb
import pandas as pd
import yvis

In [3]:
# dsess = dolphindb.session()
# dsess.connect("localhost", 8848, 'admin', 'sts.ec6.sts.130')

True

In [2]:
api = sj.Shioaji()

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [3]:
with open('login.json', 'r') as f:
    login_kwargs = json.loads(f.read())

In [ ]:
api.login(**login_kwargs, hashed=True, contracts_cb=print)

In [5]:
c = api.Contracts.Stocks['2330']
c

Stock(exchange=<Exchange.TSE: 'TSE'>, code='2330', symbol='TSE2330', name='台積電', category='24', limit_up=280.5, limit_down=229.5, reference=255.0, update_date='2020/03/24', day_trade=<DayTrade.Yes: 'Yes'>)

In [6]:
ticks_data = api.ticks(c, date='2020-03-20', timeout=100000)

In [7]:
df_ticks = pd.DataFrame({**ticks_data})

In [8]:
df_ticks['ts'] = pd.to_datetime(df_ticks['ts'])

In [9]:
df_ticks.iloc[-1]

close                         270
ask_volume                   2795
bid_volume                    645
ask_price                     270
bid_price                   269.5
ts            2020-03-20 14:30:00
volume                       1242
Name: 3167, dtype: object

In [10]:
chart = yvis.PVChart(date=df_ticks.ts.astype(str).tolist(), price=df_ticks.close.tolist(), volume=df_ticks.volume.tolist())

In [11]:
chart.content.option.tooltip

Etooltip(trigger=<EtooltipTrigger.axis: 'axis'>, axisPointer=EaxisPoint(type=<EaxisPointType.cross: 'cross'>), position=['90%', '10%'])

In [8]:
fc = sorted([txf for txf in api.Contracts.Futures.TXF], 
            key=lambda x: x.delivery_month)[0]
fc

Future(code='TXFD0', symbol='TXF202004', name='臺股期貨', category='TXF', delivery_month='202004', underlying_kind='I', limit_up=9531.0, limit_down=7799.0, reference=8665.0, update_date='2020/03/24')

In [63]:
datetime.date.today().strftime("%Y-%m-%d")

'2020-03-24'

In [ ]:
ticks_data = api.ticks(fc, date=datetime.date.today().strftime("%Y-%m-%d"), timeout=0)#100000)

In [64]:
ticks_data

Ticks(ts=[], close=[], volume=[], bid_price=[], bid_volume=[], ask_price=[], ask_volume=[])

In [60]:
df_ticks = pd.DataFrame({**ticks_data})

In [61]:
df_ticks

,volume,ask_price,ts,close,bid_price,bid_volume,ask_volume


In [9]:
from collections import defaultdict
class QuoteCollector:
    def __init__(self):
        self.quotes = defaultdict(list)
    
    def append_quote(self, topic: str, quote: dict):
        self.quotes[topic].append(quote)

In [10]:
quote_collector = QuoteCollector()

In [11]:
api.quote.set_quote_callback(quote_collector.append_quote)

In [12]:
api.quote.subscribe(fc)

Response Code: 200 | Event Code: 16 | Info: L/*/TXFD0 | Event: Subscribe or Unsubscribe ok


In [13]:
api.quote.subscribe(fc, 'bidask')

Response Code: 200 | Event Code: 16 | Info: Q/*/TXFD0 | Event: Subscribe or Unsubscribe ok


In [16]:
ticks = quote_collector.quotes[f'L/TFE/{fc.code}']

In [32]:
len(ticks)

573

In [45]:
close = [t['Close'][0] for t in  ticks]

In [46]:
volume = [t['Volume'][0] for t in  ticks]

In [47]:
dt = [t['Date'].replace('/', '-') + ' ' + t['Time'][:-3] for t in  ticks]

In [48]:
st = 0
step = 10
end = st + step
chart = yvis.PVChart(date=dt[st:end], price=close[st:end], volume=volume[st:end])

In [52]:
# chart.content.option.dataset[0].source

In [50]:
for i in range(1, 20):
    st = i * step
    end = st + step
    chart.update(date=dt[st:end], price=close[st:end], volume=volume[st:end])
    #print(i, st, end)